<a href="https://colab.research.google.com/github/cagBRT/timeSeries/blob/main/10h_Multiple_Parallel_Input_and_Multi_Step_Output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multiple Parallel Input and Multi-Step Output**

**Example of MI-MSOutout**<br>
[[10 15 25]<br>
[20 25 45]<br>
[30 35 65]<br>
[40 45 85]<br>
[50 55 105]<br>
[60 65 125]<br>
[70 75 145]<br>
[80 85 165]<br>
[90 95 185]]<br>

**If we input**:<br>
10, 15, 25 <br>
20, 25, 45 <br>
30,35, 65 <br>

**Output will be**: <br>
40, 45, 85<br>
50, 55, 105<br>

In [ ]:
def split_sequences(sequences, n_steps_in, n_steps_out):
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps_in
    out_end_ix = end_ix + n_steps_out
    # check if we are beyond the dataset
    if out_end_ix > len(sequences):
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

In [ ]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils.vis_utils import plot_model

In [ ]:
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [ ]:
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

In [ ]:
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2

In [ ]:
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
print(X.shape)
print(y.shape)

In [ ]:
# summarize the data
for i in range(len(X)):
  print(X[i], y[i])

In [ ]:
#flatten the output
n_output = y.shape[1] * y.shape[2]
y = y.reshape((y.shape[0], n_output))
n_features = X.shape[2]

In [ ]:
model = Sequential()
model.add(Conv1D(64, 2, activation='relu', input_shape=(n_steps_in, n_features))) 
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(n_output))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
model.fit(X, y, epochs=7000, verbose=0)
# demonstrate prediction
x_input = array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
x_input = x_input.reshape((1, n_steps_in, n_features))

**Make a prediction**<br>
**We expect:**<br>
90, 95, 185<br>
100, 105, 205

In [ ]:
yhat = model.predict(x_input, verbose=0)
print(yhat)

**Assignment**:<br>
1. Predict for several time steps out <br>
2. Change the output sequence equation, how does the model perform now?
